In [7]:
import pandas as pd
from datasets import DatasetDict, Dataset
import re

d:\Backend_insurance\insurance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data = pd.read_excel(r"D:\jincheng_project\Analysiss.xlsx")

In [9]:

# Remove special chars
data["title"] = data["title"].apply(lambda x: re.sub(r'[\u200b-\u200f\u202a-\u202e]', '', str(x)))
data["content"] = data["content"].apply(lambda x: re.sub(r'[\u200b-\u200f\u202a-\u202e]', '', str(x)))

# --- Define a PROMPT TEMPLATE ---
def format_question(row):
    return (
        f"根据以下信息回答问题：\n"
        f"标题: {row['title']}\n"
        f"内容: {row['content']}\n"
        f"问题类型: {row['type_name']}\n"
        f"地区: {row['af_address']}\n"
        f"请问如何处理此问题？"
    )

# Apply formatting
data["question"] = data.apply(format_question, axis=1)  # Question
data["response"] = data["jian_cha_info"]               # Answer

# Drop missing outputs
data = data.dropna(subset=["response"])  # or fill with "无回复"

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(data[["question", "response"]])
# dataset = dataset.train_test_split(test_size=0, seed=42)

In [10]:
formatted_data = pd.DataFrame({
    'Question': dataset['question'],
    'Response': dataset['response']
})

# Convert to HuggingFace Dataset
formatted_dataset = Dataset.from_pandas(formatted_data)

# Create DatasetDict with only train split
final_dataset = DatasetDict({
    'train': formatted_dataset
})

# Verify the structure
print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Response'],
        num_rows: 37624
    })
})


In [11]:
from datasets import load_from_disk, DatasetDict

# Assuming final_dataset is your DatasetDict and you want to save it to a folder named 'my_dataset_folder'
save_path = 'dataset'

# Save the dataset to the specified folder
final_dataset.save_to_disk(save_path)

print(f"Dataset saved to {save_path}")


Saving the dataset (1/1 shards): 100%|██████████| 37624/37624 [00:00<00:00, 2745129.14 examples/s]

Dataset saved to dataset
